# 🔍 KITTI Dataset Exploration

Explore the KITTI dataset structure and understand the data format for camera-LiDAR fusion.

## Dataset Overview

The KITTI dataset provides:
- 📷 **Stereo camera images** (RGB, 1241x376 pixels)
- 🎯 **LiDAR point clouds** (Velodyne HDL-64E, ~150K points/frame)
- 📐 **Calibration matrices** (Camera-LiDAR transformation)
- 🏷️ **Ground truth labels** (3D bounding boxes)

In [ ]:
# Setup environment
!git clone https://github.com/cclasher916-spec/camera-lidar-fusion.git
import os
os.chdir('/content/camera-lidar-fusion')
!pip install -q -r requirements.txt

import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path

from src.data.data_loader import KITTIDataLoader
from src.data.preprocessor import DataPreprocessor

print('✅ Environment setup complete!')

In [ ]:
# Initialize data loader
config = {
    'dataset_path': 'data/kitti',
    'sequence': '0001',
    'max_samples': 10
}

data_loader = KITTIDataLoader(config)

# Get dataset information
info = data_loader.get_sequence_info()
print('📊 Dataset Information:')
for key, value in info.items():
    print(f'  {key}: {value}')

In [ ]:
# Load and examine a sample
sample = data_loader.load_sample(0)

print('📋 Sample Structure:')
for key, value in sample.items():
    if key == 'camera_image':
        print(f'  {key}: shape {value.shape}, dtype {value.dtype}')
    elif key == 'lidar_points':
        print(f'  {key}: shape {value.shape}, {len(value)} points')
    elif key == 'calibration':
        print(f'  {key}: {len(value)} matrices')
        for k in value.keys():
            print(f'    - {k}: {value[k].shape}')
    elif key == 'ground_truth':
        print(f'  {key}: {len(value)} objects')
    else:
        print(f'  {key}: {value}')

In [ ]:
# Visualize camera image
plt.figure(figsize=(15, 5))

plt.subplot(1, 2, 1)
plt.imshow(sample['camera_image'])
plt.title('📷 Camera Image')
plt.axis('off')

# Show image histogram
plt.subplot(1, 2, 2)
for i, color in enumerate(['red', 'green', 'blue']):
    hist = cv2.calcHist([sample['camera_image']], [i], None, [256], [0, 256])
    plt.plot(hist, color=color, alpha=0.7)
plt.title('RGB Histogram')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Analyze LiDAR point cloud
points = sample['lidar_points']

print('🎯 LiDAR Point Cloud Analysis:')
print(f'  Total points: {len(points):,}')
print(f'  Point format: [x, y, z, intensity]')
print(f'  X range: [{points[:, 0].min():.2f}, {points[:, 0].max():.2f}] meters')
print(f'  Y range: [{points[:, 1].min():.2f}, {points[:, 1].max():.2f}] meters')
print(f'  Z range: [{points[:, 2].min():.2f}, {points[:, 2].max():.2f}] meters')
print(f'  Intensity range: [{points[:, 3].min():.2f}, {points[:, 3].max():.2f}]')

# Visualize point cloud statistics
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# Top-down view
ax1.scatter(points[:, 0], points[:, 1], c=points[:, 2], s=0.1, cmap='viridis')
ax1.set_xlabel('X (meters)')
ax1.set_ylabel('Y (meters)')
ax1.set_title('Top-down View (colored by height)')
ax1.axis('equal')

# Side view
ax2.scatter(points[:, 0], points[:, 2], c=points[:, 3], s=0.1, cmap='plasma')
ax2.set_xlabel('X (meters)')
ax2.set_ylabel('Z (meters)')
ax2.set_title('Side View (colored by intensity)')

# Distance distribution
distances = np.sqrt(points[:, 0]**2 + points[:, 1]**2)
ax3.hist(distances, bins=50, alpha=0.7, edgecolor='black')
ax3.set_xlabel('Distance from sensor (meters)')
ax3.set_ylabel('Point count')
ax3.set_title('Distance Distribution')

# Height distribution
ax4.hist(points[:, 2], bins=50, alpha=0.7, edgecolor='black')
ax4.set_xlabel('Height (meters)')
ax4.set_ylabel('Point count')
ax4.set_title('Height Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# Examine calibration matrices
calibration = sample['calibration']

print('📐 Calibration Matrices:')

if 'P2' in calibration:
    P2 = calibration['P2']
    print('Camera Projection Matrix (P2):')
    print(P2)
    
    # Extract camera parameters
    fx, fy = P2[0, 0], P2[1, 1]
    cx, cy = P2[0, 2], P2[1, 2]
    print(f'
Camera Parameters:')
    print(f'  Focal length: fx={fx:.1f}, fy={fy:.1f}')
    print(f'  Principal point: cx={cx:.1f}, cy={cy:.1f}')

if 'Tr_velo_to_cam' in calibration:
    Tr = calibration['Tr_velo_to_cam']
    print('
LiDAR to Camera Transformation:')
    print(Tr)

if 'R0_rect' in calibration:
    R0 = calibration['R0_rect']
    print('
Rectification Matrix:')
    print(R0)

In [ ]:
# Examine ground truth annotations
gt = sample['ground_truth']

print('🏷️ Ground Truth Annotations:')
print(f'  Number of objects: {len(gt)}')

for i, obj in enumerate(gt):
    print(f'
  Object {i+1}:')
    print(f'    Type: {obj["type"]}')
    if 'bbox_2d' in obj:
        bbox_2d = obj['bbox_2d']
        print(f'    2D BBox: [{bbox_2d[0]:.1f}, {bbox_2d[1]:.1f}, {bbox_2d[2]:.1f}, {bbox_2d[3]:.1f}]')
    if 'bbox_3d' in obj:
        bbox_3d = obj['bbox_3d']
        print(f'    3D Center: [{bbox_3d[0]:.1f}, {bbox_3d[1]:.1f}, {bbox_3d[2]:.1f}]')
        if len(bbox_3d) > 3:
            print(f'    3D Size: [{bbox_3d[3]:.1f}, {bbox_3d[4]:.1f}, {bbox_3d[5]:.1f}]')

In [ ]:
# Visualize ground truth on image
img_with_gt = sample['camera_image'].copy()

for obj in gt:
    if 'bbox_2d' in obj:
        bbox = obj['bbox_2d']
        x1, y1, x2, y2 = [int(x) for x in bbox]
        
        # Draw bounding box
        cv2.rectangle(img_with_gt, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
        # Draw label
        label = obj['type']
        cv2.putText(img_with_gt, label, (x1, y1-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

plt.figure(figsize=(15, 6))
plt.imshow(img_with_gt)
plt.title('📷 Camera Image with Ground Truth Annotations')
plt.axis('off')
plt.show()

In [ ]:
# Create bird's eye view visualization
preprocessor_config = {
    'lidar_range': {
        'x': [-50, 50],
        'y': [-25, 25],
        'z': [-3, 5]
    }
}

preprocessor = DataPreprocessor(preprocessor_config)
bev_image = preprocessor.create_bird_eye_view(points)

plt.figure(figsize=(12, 8))
plt.imshow(bev_image)
plt.title("🎯 LiDAR Bird's Eye View\n(Red: Height, Green: Intensity, Blue: Density)")
plt.xlabel('X direction (forward)')
plt.ylabel('Y direction (left)')

# Add scale information
height, width = bev_image.shape[:2]
x_range = preprocessor_config['lidar_range']['x']
y_range = preprocessor_config['lidar_range']['y']

plt.xticks([0, width//4, width//2, 3*width//4, width-1], 
          [f'{x:.0f}m' for x in np.linspace(x_range[0], x_range[1], 5)])
plt.yticks([0, height//4, height//2, 3*height//4, height-1],
          [f'{y:.0f}m' for y in np.linspace(y_range[1], y_range[0], 5)])

plt.show()

In [ ]:
# Compare multiple samples
num_samples = min(4, data_loader.get_sample_count())

fig, axes = plt.subplots(2, num_samples, figsize=(20, 8))
if num_samples == 1:
    axes = axes.reshape(2, 1)

for i in range(num_samples):
    sample = data_loader.load_sample(i)
    
    # Camera image
    axes[0, i].imshow(sample['camera_image'])
    axes[0, i].set_title(f'Sample {i} - Camera')
    axes[0, i].axis('off')
    
    # LiDAR bird's eye view
    bev = preprocessor.create_bird_eye_view(sample['lidar_points'])
    axes[1, i].imshow(bev)
    axes[1, i].set_title(f'Sample {i} - LiDAR BEV')
    axes[1, i].axis('off')

plt.suptitle('📊 Multi-Sample Comparison', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
# Data preprocessing analysis
original_points = sample['lidar_points']
processed_points = preprocessor.preprocess_lidar(original_points)

stats = preprocessor.get_preprocessing_stats(
    sample['camera_image'], 
    original_points
)

print('🔄 Preprocessing Statistics:')
for key, value in stats.items():
    print(f'  {key}: {value}')

# Visualize filtering effects
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.scatter(original_points[:, 0], original_points[:, 1], 
           c=original_points[:, 2], s=0.1, cmap='viridis')
plt.title(f'Original Points ({len(original_points):,})')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.axis('equal')

plt.subplot(1, 3, 2)
plt.scatter(processed_points[:, 0], processed_points[:, 1], 
           c=processed_points[:, 2], s=0.1, cmap='viridis')
plt.title(f'Filtered Points ({len(processed_points):,})')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.axis('equal')

plt.subplot(1, 3, 3)
reduction = (len(original_points) - len(processed_points)) / len(original_points) * 100
plt.bar(['Original', 'Filtered'], [len(original_points), len(processed_points)], 
        color=['blue', 'orange'], alpha=0.7)
plt.title(f'Point Reduction: {reduction:.1f}%')
plt.ylabel('Number of Points')

plt.tight_layout()
plt.show()

## 📋 Summary

This exploration revealed:

### Data Characteristics
- **Camera**: High-resolution RGB images with rich semantic information
- **LiDAR**: Dense 3D point clouds with accurate geometric information
- **Calibration**: Precise sensor alignment for coordinate transformation

### Key Insights
1. **Complementary Information**: Camera provides texture/color, LiDAR provides depth
2. **Different Failure Modes**: Each sensor has unique strengths and weaknesses
3. **Preprocessing Importance**: Filtering reduces noise and computational load
4. **Ground Truth Quality**: Accurate annotations enable proper evaluation

### Fusion Motivation
The exploration confirms that camera and LiDAR sensors provide complementary information that can be fused to create a more robust perception system than either sensor alone.